In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score
import optuna
import ast
import math 
import matplotlib.pyplot as plt
import scipy.stats as stats


In [15]:
train=pd.read_csv("train.csv", sep=';')
test=pd.read_csv("test.csv", sep=';')
refer_vectors=pd.read_csv("referer_vectors.csv", sep=';')
train_labels=pd.read_csv("train_labels.csv", sep=';')
geo_info=pd.read_csv("geo_info.csv", sep=';')

In [16]:
train = train.merge(refer_vectors, on='referer')
train = train.merge(train_labels, on='user_id')
train = train.merge(geo_info, on='geo_id')


In [17]:
def safe_literal_eval(x):
    if pd.isna(x):
        return {}
    try:
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        return {}

# Извлечение информации о браузере, ОС и их версиях
train['parsed_user_agent'] = train['user_agent'].apply(safe_literal_eval)

train['browser'] = train['parsed_user_agent'].apply(lambda x: x.get('browser', None))
train['browser_version'] = train['parsed_user_agent'].apply(lambda x: x.get('browser_version', None))
train['os'] = train['parsed_user_agent'].apply(lambda x: x.get('os', None))
train['os_version'] = train['parsed_user_agent'].apply(lambda x: x.get('os_version', None))

train.drop(columns=['parsed_user_agent'], inplace=True)

In [18]:
test = test.merge(refer_vectors, on='referer')
test = test.merge(train_labels, on='user_id')
test = test.merge(geo_info, on='geo_id')


In [19]:
test['parsed_user_agent'] = test['user_agent'].apply(safe_literal_eval)

test['browser'] = test['parsed_user_agent'].apply(lambda x: x.get('browser', None))
test['browser_version'] = test['parsed_user_agent'].apply(lambda x: x.get('browser_version', None))
test['os'] = test['parsed_user_agent'].apply(lambda x: x.get('os', None))
test['os_version'] = test['parsed_user_agent'].apply(lambda x: x.get('os_version', None))

test.drop(columns=['parsed_user_agent'], inplace=True)

In [20]:
train.columns

Index(['request_ts', 'user_id', 'referer', 'geo_id', 'user_agent',
       'component0', 'component1', 'component2', 'component3', 'component4',
       'component5', 'component6', 'component7', 'component8', 'component9',
       'target', 'country_id', 'region_id', 'timezone_id', 'browser',
       'browser_version', 'os', 'os_version'],
      dtype='object')

In [21]:
test.columns

Index(['request_ts', 'user_id', 'referer', 'geo_id', 'user_agent',
       'component0', 'component1', 'component2', 'component3', 'component4',
       'component5', 'component6', 'component7', 'component8', 'component9',
       'target', 'country_id', 'region_id', 'timezone_id', 'browser',
       'browser_version', 'os', 'os_version'],
      dtype='object')

In [32]:
train=train.dropna()

In [33]:
X = train.drop(columns=['target', 'user_id', 'referer', 'request_ts', 'user_agent', 'os_version'])
y = train['target']

categorical_features = ['country_id', 'region_id',  'timezone_id','geo_id', 'browser', 'browser_version', 'os']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


# Обучение модели CatBoost

In [34]:
model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.1,
                           depth=6,
                           random_seed=42,
                           loss_function='Logloss',
                           cat_features=categorical_features,
                           verbose=100)

model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50)


0:	learn: 0.6669823	test: 0.6672614	best: 0.6672614 (0)	total: 377ms	remaining: 6m 16s
100:	learn: 0.4806411	test: 0.4829955	best: 0.4829955 (100)	total: 26.4s	remaining: 3m 54s
200:	learn: 0.4625617	test: 0.4653501	best: 0.4653501 (200)	total: 52s	remaining: 3m 26s
300:	learn: 0.4529549	test: 0.4564817	best: 0.4564817 (300)	total: 1m 18s	remaining: 3m 2s
400:	learn: 0.4471573	test: 0.4516279	best: 0.4516279 (400)	total: 1m 44s	remaining: 2m 35s
500:	learn: 0.4430343	test: 0.4485071	best: 0.4485071 (500)	total: 2m 10s	remaining: 2m 10s
600:	learn: 0.4396597	test: 0.4460071	best: 0.4460071 (600)	total: 2m 36s	remaining: 1m 44s
700:	learn: 0.4367987	test: 0.4443084	best: 0.4443084 (700)	total: 3m 2s	remaining: 1m 18s
800:	learn: 0.4342850	test: 0.4428542	best: 0.4428542 (800)	total: 3m 28s	remaining: 51.9s
900:	learn: 0.4321073	test: 0.4415791	best: 0.4415791 (900)	total: 3m 54s	remaining: 25.8s
999:	learn: 0.4301764	test: 0.4404812	best: 0.4404812 (999)	total: 4m 19s	remaining: 0us

bes

# Метрики 

In [39]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

y_test = test['target'] 
X_test = test.drop(columns=['target', 'user_id', 'referer', 'request_ts', 'user_agent', 'os_version'])
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1] 

# Вычисление метрик
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

# Вывод результатов
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'ROC AUC: {roc_auc:.4f}')



Accuracy: 0.3367
Precision: 0.3552
Recall: 0.3244
F1 Score: 0.3391
ROC AUC: 0.3202


# Подберём гиперпараметры оптуной

In [42]:
def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 4, 10),
        'loss_function': 'Logloss',
        'cat_features': categorical_features,
        'random_seed': 42,
        'verbose': 0
    }

    model = CatBoostClassifier(**params)
    model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50, verbose=0)

    y_pred_proba = model.predict_proba(X_val)[:, 1]
    logloss = log_loss(y_val, y_pred_proba)

    return logloss


In [41]:
study = optuna.create_study(direction='minimize') 
study.optimize(objective, n_trials=5) 

[I 2024-12-10 14:13:39,075] A new study created in memory with name: no-name-704f3b46-b23c-43bd-9943-89a0e13c8ddb
[W 2024-12-10 14:16:24,471] Trial 0 failed with parameters: {'iterations': 962, 'learning_rate': 0.20967927234222958, 'depth': 9} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "/home/arhipov.danil7/.pyenv/versions/3.10.13/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_2014/406187875.py", line 14, in objective
    model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50, verbose=0)
  File "/home/arhipov.danil7/.pyenv/versions/3.10.13/lib/python3.10/site-packages/catboost/core.py", line 5220, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline, use_best_model,
  File "/home/arhipov.danil7/.pyenv/versions/3.10.13/lib/python3.10/site-package

KeyboardInterrupt: 

## Попробуем использовать random_forest

In [47]:
train_for_rf=train.drop(columns=['country_id', 'region_id',  'timezone_id','geo_id', 'browser', 'browser_version', 'os'])

In [48]:
X = train_for_rf.drop(columns=['target', 'user_id', 'referer', 'request_ts', 'user_agent', 'os_version'])
y = train_for_rf['target']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [49]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

rf_model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [50]:
test_for_rf=test.drop(columns=['country_id', 'region_id',  'timezone_id','geo_id', 'browser', 'browser_version', 'os'])

In [52]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

y_test = test_for_rf['target'] 
X_test = test_for_rf.drop(columns=['target', 'user_id', 'referer', 'request_ts', 'user_agent', 'os_version'])
y_pred = rf_model.predict(X_test)
y_pred_proba = rf_model.predict_proba(X_test)[:, 1] 

# Вычисление метрик
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

# Вывод результатов
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'ROC AUC: {roc_auc:.4f}')



Accuracy: 0.4747
Precision: 0.4993
Recall: 0.4854
F1 Score: 0.4922
ROC AUC: 0.4845


# Выводы


1. Используя Catboost мы получили следующие результаты
   
Accuracy (Точность): 0.3367 <br>
Точность модели составляет 33.67%, что довольно низкий показатель. <br>
Это может указывать на дисбаланс классов, когда один класс (например, мужчины) преобладает над другим. <br>
Precision (Точность положительных предсказаний): 0.3552 <br>
Это означает, что 35.52% предсказанных положительных случаев (например, женский пол) действительно являются верными. Точность относительно низкая. <br>
Recall (Полнота): 0.3244 <br>
Полнота показывает, что модель правильно предсказала 32.44% всех реальных положительных случаев (например, женщин). Это также достаточно низкий показатель, <br>что говорит о том, что модель не выявляет достаточно большого числа реальных положительных примеров. <br>
2. Рассмотрим метрики, полученные для модели случайного леса (Random Forest):
Accuracy (Точность): 0.4747
Точность составляет 47.47%. Это улучшение по сравнению с предыдущей моделью, но все еще менее половины правильных предсказаний. 
Precision (Точность положительных предсказаний): 0.4993
Это означает, что 49.93% предсказанных положительных случаев (например, женщин) действительно являются верными. Модель достаточно хорошо идентифицирует положительные примеры.
Recall (Полнота): 0.4854
Полнота равна 48.54%, что говорит о том, что модель правильно классифицировала 48.54% всех реальных положительных случаев.
